### Limpeza dos dados carregados

- Carregar ambos os CSVs, cliente e vendas
- Indentificar ponto de junção
- Fazer limpeza e transformação para a junção
- Retirada de variáveis desnecessárias
- Criação de novas colunas (Nome, E-mail)*******

### Informações sobre esse notebook

Aqui será feito a limpeza dos dados carregados do Kaggle. A meta é fazer a junção de dois datasets, um é relacionado a vendas de produtos (e-commerce) e o outro são dados dos clientes que fizeram essas compras. O objetivo desse processo é tentar fazer com que os dados sejam os mais próximos possíveis do que possam haver em um banco de dados de uma empresa de e-commerce.

Por serem dados ficticios poderão ter alguns erros de incoerencia.

Os dados limpos após a transformação para poder fazer a junção estão salvos com o nome 'clientes_info_limpo.csv' e 'vendas_info_limpo.csv'.

#### Importação das bibliotecas

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from faker import Faker

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

Importação dos dados que serão utilizados

In [6]:
dados_vendas = os.getenv("ENDERECO_DADOS_VENDA")
dados_clientes = os.getenv("ENDERECO_DADOS_CLIENTES")
df_vendas = pd.read_csv(dados_vendas, encoding='latin-1')
df_clientes = pd.read_csv(dados_clientes)

Visualização do que temos no primeiro dataframe relacionado com as vendas.

In [7]:
df_vendas.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Visualização dos dados relacionados aos clientes.

In [8]:
df_clientes.head()

,Unnamed: 0,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,...,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,...,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,159.93
1,1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,...,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,120.90
2,2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,...,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120.28
3,3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,...,Laptop & Accessory,589314,Single,8,0,23.0,0.0,1.0,3.0,134.07
4,4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,...,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,129.60


##### Observações
Podemos perceber que temos que fazer alterações em algumas informações como trocar o nome os CustomerID para fazer relação com a outra tabela, além de retirar algumas colunas desnecessário e que pode acarretar conflito com o outro dataframe.

Vamos visualizar abaixo quantos valores únicos de IDs há em cada um dos DataFrames.

In [9]:
print(f'Quantidade de valores únicos na tabela clientes na coluna CustomerID:\n{df_clientes.CustomerID.nunique()}')
print(f'Quantidade de valores únicos na tabela vandas na coluna CustomerID:\n{df_vendas.CustomerID.nunique()}')

Quantidade de valores únicos na tabela clientes na coluna CustomerID:
5630
Quantidade de valores únicos na tabela vandas na coluna CustomerID:
4372


Como podemos ver a cima temos mais IDs em um dataframe que em outro. Porem para que fiquem ajustados para futuros processos iremos retirar algumas linhas da tabela cliente para ficar de acordo com a tabela de vendas. 

Isso será secessário nesse caso pois é um adequação a valores fictícios na qual precisamos de ambas as informações. Lembrando que o objetivo é simular dados disponíveis em um banco de dados de uma empresa de e-commerce.

Vamos ver abaixo qual a diferença entre ele para que possamos retirar da tabela clientes.

In [10]:
print(5630-4372)

1258


Antes de retirarmos da tabela, vamos se nela há valores duplicados. E como foi possível ver temos uma boa parte de dados duplicados com base no ID do cliente.

In [11]:
df_clientes.duplicated('CustomerID').value_counts()

True     11260
False     5630
Name: count, dtype: int64

Com isso, vamos retirar esses dados duplicados do nosso dataframe.

In [12]:
df_clientes_sem_duplicatas = df_clientes.drop_duplicates('CustomerID')
df_clientes_sem_duplicatas

,Unnamed: 0,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,...,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,...,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,159.93
1,1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,...,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,120.90
2,2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,...,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120.28
3,3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,...,Laptop & Accessory,589314,Single,8,0,23.0,0.0,1.0,3.0,134.07
4,4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,...,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,129.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,5625,55626,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,...,Laptop & Accessory,1,Married,6,0,18.0,1.0,2.0,4.0,150.71
5626,5626,55627,0,13.0,Mobile Phone,1,13.0,Credit Card,Male,3.0,...,Fashion,5,Married,6,0,16.0,1.0,2.0,NaN,224.91
5627,5627,55628,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,...,Laptop & Accessory,4,Married,3,1,21.0,1.0,2.0,4.0,186.42
5628,5628,55629,0,23.0,Computer,3,NaN,Credit Card,Male,4.0,...,Laptop & Accessory,4,Married,4,0,15.0,2.0,2.0,9.0,178.90


Agora precisamos remorar os 1257 IDs que estão excedentes em relação ao outro dataframe. Irei fazer isso de maneira aleatória, assim não teremos uma perda direcionda aos ultimos clientes acrescentados.

In [13]:
df_clientes_reduzido = df_clientes_sem_duplicatas.drop(df_clientes_sem_duplicatas.sample(n=1257, random_state=42).index)
df_clientes_reduzido


,Unnamed: 0,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,...,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,...,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,159.93
1,1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,...,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,120.90
2,2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,...,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120.28
3,3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,...,Laptop & Accessory,589314,Single,8,0,23.0,0.0,1.0,3.0,134.07
4,4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,...,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,129.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,5625,55626,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,...,Laptop & Accessory,1,Married,6,0,18.0,1.0,2.0,4.0,150.71
5626,5626,55627,0,13.0,Mobile Phone,1,13.0,Credit Card,Male,3.0,...,Fashion,5,Married,6,0,16.0,1.0,2.0,NaN,224.91
5627,5627,55628,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,...,Laptop & Accessory,4,Married,3,1,21.0,1.0,2.0,4.0,186.42
5628,5628,55629,0,23.0,Computer,3,NaN,Credit Card,Male,4.0,...,Laptop & Accessory,4,Married,4,0,15.0,2.0,2.0,9.0,178.90


Podemos fazer agora a transformação dos IDs do cliente com base nos IDs do dataframe de venda.

In [14]:
df_clientes_reduzido['CustomerID'] = df_vendas['CustomerID'].unique()
df_clientes_reduzido

,Unnamed: 0,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,...,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,0,17850.0,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,...,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,159.93
1,1,13047.0,1,NaN,Phone,1,8.0,UPI,Male,3.0,...,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,120.90
2,2,12583.0,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,...,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120.28
3,3,13748.0,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,...,Laptop & Accessory,589314,Single,8,0,23.0,0.0,1.0,3.0,134.07
4,4,15100.0,1,0.0,Phone,1,12.0,CC,Male,NaN,...,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,129.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,5625,13436.0,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,...,Laptop & Accessory,1,Married,6,0,18.0,1.0,2.0,4.0,150.71
5626,5626,15520.0,0,13.0,Mobile Phone,1,13.0,Credit Card,Male,3.0,...,Fashion,5,Married,6,0,16.0,1.0,2.0,NaN,224.91
5627,5627,13298.0,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,...,Laptop & Accessory,4,Married,3,1,21.0,1.0,2.0,4.0,186.42
5628,5628,14569.0,0,23.0,Computer,3,NaN,Credit Card,Male,4.0,...,Laptop & Accessory,4,Married,4,0,15.0,2.0,2.0,9.0,178.90


Outro ponto necessário agora é a retirada de informações que não irão acrescentar e também podem ser prejudicial. Iremos remover algumas colunas.

In [15]:
colunas_remover = ['Unnamed: 0', 'CityTier','WarehouseToHome','HourSpendOnApp', 
                   'PreferedOrderCat','NumberOfAddress', 'OrderAmountHikeFromlastYear', 
                   'OrderCount', 'DaySinceLastOrder', 'CashbackAmount']

df_clientes_reduzido.drop(columns=colunas_remover, inplace=True)
df_clientes_reduzido.head()

,CustomerID,Churn,Tenure,PreferredLoginDevice,PreferredPaymentMode,Gender,NumberOfDeviceRegistered,SatisfactionScore,MaritalStatus,Complain,CouponUsed
0,17850.0,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0
1,13047.0,1,NaN,Phone,UPI,Male,4.0,3,Single,1,0.0
2,12583.0,1,NaN,Phone,Debit Card,Male,4.0,3,Single,1,0.0
3,13748.0,1,0.0,Phone,Debit Card,Male,4.0,589314,Single,0,0.0
4,15100.0,1,0.0,Phone,CC,Male,3.0,5,Single,0,1.0


Para saber se foi feito corretamente e os dados estão conectados, irei criar uma tabela que fazer a junção de ambas.

In [16]:
df_merged = df_vendas.merge(df_clientes_reduzido, on='CustomerID', how='inner')  
df_merged


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Churn,Tenure,PreferredLoginDevice,PreferredPaymentMode,Gender,NumberOfDeviceRegistered,SatisfactionScore,MaritalStatus,Complain,CouponUsed
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,0,28.0,Mobile Phone,Cash on Delivery,Female,4.0,3,Married,0,1.0
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,0,28.0,Mobile Phone,Cash on Delivery,Female,4.0,3,Married,0,1.0
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,0,28.0,Mobile Phone,Cash on Delivery,Female,4.0,3,Married,0,1.0
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,0,28.0,Mobile Phone,Cash on Delivery,Female,4.0,3,Married,0,1.0


#### Limpeza dos dados brutos - Vendas

Essa limpeza esta relacionada ao que será acrescentado na transformação do processo de ETL.

In [17]:
dados_sem_null = df_vendas.dropna(subset=['CustomerID'])
df_vendas_limpo = dados_sem_null[(dados_sem_null['Quantity'] > 0) & (dados_sem_null['UnitPrice'] > 0)]
df_vendas_limpo['InvoiceDate'] = pd.to_datetime(df_vendas_limpo['InvoiceDate'])

C:\Users\heloi\AppData\Local\Temp\ipykernel_14096\986452201.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vendas_limpo['InvoiceDate'] = pd.to_datetime(df_vendas_limpo['InvoiceDate'])


In [18]:
df_vendas_limpo

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


### Acrescentado colunas importantes para o projeto
- Nome
- E-mail

In [19]:
df_clientes_reduzido.head()

,CustomerID,Churn,Tenure,PreferredLoginDevice,PreferredPaymentMode,Gender,NumberOfDeviceRegistered,SatisfactionScore,MaritalStatus,Complain,CouponUsed
0,17850.0,1,4.0,Mobile Phone,Debit Card,Female,3.0,2,Single,1,1.0
1,13047.0,1,NaN,Phone,UPI,Male,4.0,3,Single,1,0.0
2,12583.0,1,NaN,Phone,Debit Card,Male,4.0,3,Single,1,0.0
3,13748.0,1,0.0,Phone,Debit Card,Male,4.0,589314,Single,0,0.0
4,15100.0,1,0.0,Phone,CC,Male,3.0,5,Single,0,1.0


Irei salvar os novos dados transformados e limpos para que seja possível utilizar em outras partes desse processo.

In [20]:
df_clientes_reduzido.to_csv(os.getenv("ENDERECO_DADOS_LIMPOS_CLIENTES"), index=False)
df_vendas_limpo.to_csv(os.getenv("ENDERECO_DADOS_LIMPOS_VENDAS"), index=False)